In [6]:
import numpy as np
import math 
from qiskit import *
#from qiskit.visualization import plot_histogram
from qiskit.circuit import Parameter
from math import pi
import operator
%matplotlib inline
import random

n  = 2
dim = 2
nodes = [[0 for l in range(dim)] for m in range(dim)] 
literals = {} # use key-value pair
grid = [[0 for s in range(dim)] for t in range(dim)] 
func = ["xor","xor","xor","xor","xor","xor"]
opt = 0.0
P = 8 
graphX = [0 for s in range(P)]
graphY = [0.0 for s in range(P)]
operator = func[random.randint(0,len(func)-1)]

def check_neighbors(i, j):
    if (j - 1) > -1:
        if nodes[i][j-1] == -1:
            return 0
    elif (j-1) == -1:
        if nodes[i][dim-1] == -1:
            return 0
        
    if (j + 1) < dim:
        if nodes[i][j+1] == -1:
            return 0
    elif (j+1) == dim:
        if nodes[i][0] == -1:
            return 0
        
    if (i - 1) > -1:
        if nodes[i-1][j] == -1:
            return 0
    elif (i-1) == -1:
        if nodes[dim-1][j] == -1:
            return 0
        
    if (i + 1) < dim:
        if nodes[i+1][j] == -1:
            return 0
    elif (i + 1) == dim:
        if nodes[0][j] == -1:
            return 0
        
    return 1

def qaoa(angle0,angle1,nodes):
    theta = Parameter('θ')
    gamma = Parameter('γ')
    theta1 = Parameter('θ1')
    gamma1 = Parameter('γ1')
    #angle0 = (np.pi)/2
    #angle1= (np.pi)
    
    q = QuantumRegister(1)

    qc = QuantumCircuit(n*n,n*n)

    #convert to all 1s ### init qubits to node values
    #for i in range(n):
    #   qc.x(i)
    #qc.barrier()

#Conditional X... Apply B

    #qc.ccx(1,2,3)
    #qc.cu3(gamma,pi/2,pi/2,3,0) #cu3(gamma,pi/2,pi/2,c,t) = crx(gamma,c,t)
    #qc.ccx(1,2,3)
    
    for i in range(n):
        for j in range(n):
            if AND_neighbors(i,j):
                qc.rx(gamma,i*j)
                nodes[i][j] = 1


    qc.barrier()

# Z rotation... apply C
    #qc.rz(theta, range(n))
    # #### use 
    for i in range(n*n):
        if(i == 0):
            qc.rzz(theta,(n*n - 1),i)
        if(i > 0) and (i < (n*n - 1)):
            qc.rzz(theta, i-1,i)
        if(i == (n*n - 1)):
            qc.rzz(theta, i,(n*n - 1))
    

    qc.barrier()
    
    for i in range(n*n):
        qc.measure(i, i)

    circuits = [qc.bind_parameters({theta: -(np.pi)/2,gamma: angle0,theta1: -(np.pi), gamma1: angle1})]
#circuits = [qc.bind_parameters({gamma: (np.pi)/2,gamma1: (np.pi)})]
    #circuits = [qc.bind_parameters({theta: -(np.pi),gamma: angle1})]
    #print(circuits[-1])
    circuits[-1].draw(output='mpl')

# Use Aer's qasm_simulator
    backend_sim = BasicAer.get_backend('qasm_simulator')
    backend = BasicAer.get_backend('statevector_simulator')

# Execute the circuit on the qasm simulator.
# We've set the number of repeats of the circuit
# to be 1024, which is the default.
    job_sim = execute(circuits, backend_sim, shots=1024)
# Create a Quantum Program for execution 
    job = execute(circuits, backend)

# Grab the results from the job.
    result_sim = job_sim.result()
    result = job.result()

    counts = result_sim.get_counts(qc)
    #print(counts)

    outputstate = result.get_statevector(qc, decimals=3)
    #print(outputstate)
    return counts

def hammingWeight(z):
    weight = 0
    for x in z:
        if x == '0':
            weight+= 1
    return weight

def main():
    print("We got QAOA!")
    #m = 2
    maxAmp = 0
    bestZ = ''
    bestAngles = []
    bestDistribution = {}
    runs = 0
    for i in range(1):
        for j in range(1):
            print("run:",runs)
            runs += 1
            distribution = qaoa((math.radians(90)),(math.radians(90)),nodes)
            #distribution = qaoa((math.radians(i)),(math.radians(j)))
            if(max(distribution.items(), key=operator.itemgetter(1))[1] > maxAmp):
                bestZ = max(distribution.items(), key=operator.itemgetter(1))[0]
                maxAmp = max(distribution.items(), key=operator.itemgetter(1))[1]
                bestDistribution = distribution
                bestAngles.clear()
                bestAngles.append(math.radians(i))
                bestAngles.append(math.radians(j))
    #distribution = qaoa((math.radians(90.0)),(math.radians(180.0)))
    print(bestDistribution)
    #print(bestAngles)
    plot_histogram(bestDistribution).savefig('out.png')
    print(bestZ)
    print(bestAngles)
    print(maxAmp)

               
print("operator:"+operator)
print() 

    
if __name__ == '__main__':
    main()


ModuleNotFoundError: No module named 'qiskit.circuit'